In [11]:
import pandas as pd
import os
import warnings
import datetime
from datetime import datetime
from dateutil.relativedelta import relativedelta

In [12]:
warnings.filterwarnings('ignore')

In [17]:
path_leave = r'C:\Users\charoenchai.chu\Desktop\Rules\เคสบางสะพาน_Model'
name_leave = 'ข้อมูลวันลา_3ปี'

file_path1 = os.path.join(path_leave, f'{name_leave}.xlsx')
df_leave = pd.read_excel(file_path1)
df_leave = df_leave[['โค้ด','ชือ - นามสกุล','ตำแหน่ง','วันที่เริ่มต้น','วันที่สิ้นสุด','รอบเวลา (1 วัน)']]
# df_leave.head()

In [18]:
#เปลี่ยน path
path_lone = r'C:\Users\charoenchai.chu\Desktop\Rules\เคสบางสะพาน_Model'
file_lone = 'รวมไฟล์_new'

file_path_lone = os.path.join(path_lone, file_lone + '.xlsx')
df_lone=pd.read_excel(file_path_lone , skiprows= 3)
df_lone['submitbranch']=df_lone['submitbranch'].astype(str).str.zfill(3)
# df_lone['journaltimestamp','recordtimestamp']
# df_lone.head()

In [19]:
# แปลงคอลัมน์ recordtimestamp และ journaltimestamp เป็น datetime
df_lone['recordtimestamp'] = pd.to_datetime(df_lone['recordtimestamp'], errors='coerce')
df_lone['journaltimestamp'] = pd.to_datetime(df_lone['journaltimestamp'], errors='coerce')

# ฟังก์ชันเพื่อเพิ่ม 1 ปีให้กับวันที่
def add_year(date):
    if pd.notna(date):  # ตรวจสอบว่าไม่เป็น NaT
        return date + relativedelta(years=1)
    return date

# เพิ่มวันที่ 1 ปี
df_lone['recordtimestamp'] = df_lone['recordtimestamp'].apply(add_year)
df_lone['journaltimestamp'] = df_lone['journaltimestamp'].apply(add_year)

# df_lone.head()

In [20]:
# join วันลา กับ lone
df_leave['โค้ด'] = df_leave['โค้ด'].astype(str).str.zfill(3)
df_lone['submitbranch'] = df_lone['submitbranch'].astype(str).str.zfill(3)

df_leave['วันที่เริ่มต้น'] = pd.to_datetime(df_leave['วันที่เริ่มต้น']).dt.date
df_leave['วันที่สิ้นสุด'] = pd.to_datetime(df_leave['วันที่สิ้นสุด']).dt.date
df_lone['date'] = pd.to_datetime(df_lone['recordtimestamp']).dt.date

# ฟิลเตอร์ข้อมูลในสาขาเดียวกันและวันที่ตรงกัน
data_df = df_lone.merge(df_leave, how='left', 
                           left_on=['date', 'submitbranch'], 
                           right_on=['วันที่เริ่มต้น', 'โค้ด'])
# data_df.head()

#### Set Rules

In [21]:
# มีการทำสัญญานอกเหนือจากเวลาทำงาน

# แปลงคอลัมน์ 'recordtimestamp' ให้เป็น datetime
df_lone['recordtimestamp'] = pd.to_datetime(df_lone['recordtimestamp'])

# กำหนดช่วงเวลาที่ต้องการกรอง
morning_start = pd.to_datetime('08:30:00').time()
morning_end = pd.to_datetime('12:00:00').time()
afternoon_start = pd.to_datetime('13:00:00').time()
afternoon_end = pd.to_datetime('17:00:00').time()

# ก่อน 08.30
before_morning_time = df_lone[(df_lone['recordtimestamp'].dt.time < morning_start)]

# # กรองข้อมูลที่ไม่อยู่ในช่วงเวลาทำงาน
# filtered_time = df_lone[~((df_lone['recordtimestamp'].dt.time >= morning_start) & (df_lone['recordtimestamp'].dt.time <= morning_end) |
#                     (df_lone['recordtimestamp'].dt.time >= afternoon_start) & (df_lone['recordtimestamp'].dt.time <= afternoon_end))]

# # แสดงข้อมูลที่กรองแล้ว
# # filtered_df['recordtimestamp']
before_morning_time['Rules01'] = 'มีการทำสัญญาก่อนเวลาทำงาน'
filtered_time1 = before_morning_time[['submitbranch','policyno','recordtimestamp','Rules01']]
# filtered_time1

In [22]:
# # พักเที่ยง ลันช์ไทม์
luch_time = df_lone[(df_lone['recordtimestamp'].dt.time > morning_end) & (df_lone['recordtimestamp'].dt.time < afternoon_start)]
# # หลัง 17.00
# after_afternoon_end = df_lone[(df_lone['recordtimestamp'].dt.time > afternoon_end)]

# # กรองข้อมูลที่ไม่อยู่ในช่วงเวลาที่กำหนด
# filtered_time = df_lone[~((df_lone['recordtimestamp'].dt.time >= morning_start) & (df_lone['recordtimestamp'].dt.time <= morning_end) |
#                     (df_lone['recordtimestamp'].dt.time >= afternoon_start) & (df_lone['recordtimestamp'].dt.time <= afternoon_end))]

# # แสดงข้อมูลที่กรองแล้ว
# # filtered_df['recordtimestamp']
luch_time['Rules02'] = 'มีการทำสัญญาช่วงเวลาพักเที่ยง'
filtered_time2 = luch_time[['submitbranch','policyno','recordtimestamp','Rules02']]
# filtered_time2

In [23]:
# # หลัง 17.00
after_afternoon_end = df_lone[(df_lone['recordtimestamp'].dt.time > afternoon_end)]

# # แสดงข้อมูลที่กรองแล้ว
# # filtered_df['recordtimestamp']
after_afternoon_end['Rules03'] = 'มีการทำสัญญาหลังเวลาเลิกงาน'
filtered_time3 = after_afternoon_end[['submitbranch','policyno','recordtimestamp','Rules03']]
# filtered_time3

In [24]:
# มีพนักงานคนใดคนหนึ่งลาในช่วงเวลาทำงาน
# data_df1 = data_df[['โค้ด','policyno1']]
# ฟิลเตอร์ข้อมูลในสาขาเดียวกันและวันที่ตรงกัน
filter_leave = data_df[data_df['โค้ด'].notnull()]

filter_leave['Rules04'] = 'มีการสร้างสัญญาในวันที่อยู่สาขาคนเดียว'
filter_leave[['โค้ด','policyno','Rules04']]
# filter_leave
# leave

,โค้ด,policyno,Rules04
18,032,38169239,มีการสร้างสัญญาในวันที่อยู่สาขาคนเดียว
19,032,35588533,มีการสร้างสัญญาในวันที่อยู่สาขาคนเดียว
26,032,37900822,มีการสร้างสัญญาในวันที่อยู่สาขาคนเดียว
31,032,86017028,มีการสร้างสัญญาในวันที่อยู่สาขาคนเดียว
33,032,90241056,มีการสร้างสัญญาในวันที่อยู่สาขาคนเดียว
...,...,...,...
26712,850,90178243,มีการสร้างสัญญาในวันที่อยู่สาขาคนเดียว
26725,634,90198171,มีการสร้างสัญญาในวันที่อยู่สาขาคนเดียว
26730,691,90206047,มีการสร้างสัญญาในวันที่อยู่สาขาคนเดียว
26739,634,90215564,มีการสร้างสัญญาในวันที่อยู่สาขาคนเดียว


In [25]:
# ทำ left join โดยเริ่มจาก df1 กับ df2
df_left_join_1 = data_df.merge(filtered_time1, how='left', on=['submitbranch','policyno','recordtimestamp'])
df_left_join_2 = df_left_join_1.merge(filtered_time2, how='left', on=['submitbranch','policyno','recordtimestamp'])
df_left_join_3 = df_left_join_2.merge(filtered_time3, how='left', on=['submitbranch','policyno','recordtimestamp'])

# จากนั้นทำ left join กับ df3
df_left_join_4 = df_left_join_3.merge(filter_leave, how='left', on=['โค้ด','policyno'])

# แสดงข้อมูลที่ได้

In [26]:
df_left_join_4

,policyno,prename_x,firstname_x,lastname_x,policystatus1_x,policystatusdate1_x,policyno1_x,loandate_x,Policyloandate_x,registerno_x,...,name6_y,zipcode_y,phoneno_y,date_y,ชือ - นามสกุล_y,ตำแหน่ง_y,วันที่เริ่มต้น_y,วันที่สิ้นสุด_y,รอบเวลา (1 วัน)_y,Rules04
0,35588533,000001,นิยม,เซี่ยงฉี,F,25710226,35588533,25670628,3558853325670628,35588533,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,90214035,000004,ภากร,สิงห์ลอ,I,25670625,90214035,25670625,9021403525670625,90214035,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,37900855,000002,บานเย็น,ทิมทอง,C,25670623,37900855,25670620,3790085525670620,37900855,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,23224015,000003,นิษา,สมรูป,I,25650531,23224015,25670615,2322401525670615,23224015,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,36557624,000002,พรทิพย์,พุฒิลือชา,I,0,36557624,25670614,3655762425670614,36557624,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27221,90271138,000003,วิเชียร,มาพันธุ์,I,25670116,90271138,25670318,9027113825670318,90271138,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27222,90273073,000003,ปณิตา,กลัดเจ็ด,I,25670422,90273073,25670628,9027307325670628,90273073,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27223,90274255,000001,สุทัศน์,จันทะไชย,I,25650317,90274255,25670105,9027425525670105,90274255,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27224,90278488,000003,วันดี,ยาสาธร,I,25650429,90278488,25670628,9027848825670628,90278488,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [42]:
# รับวันที่ปัจจุบันในรูปแบบที่ต้องการ
now = datetime.now().strftime("%d%m%Y")

# กำหนดพาธของโฟลเดอร์
file_path1 = r'C:\Users\charoenchai.chu\Desktop\Rules\เคสบางสะพาน'
os.makedirs(file_path1, exist_ok=True)  # สร้างโฟลเดอร์ถ้ายังไม่มี

# สร้างชื่อไฟล์ด้วยวันที่
file_name_finish = f"เคสบางสะพาน_ALL_{now}.xlsx"
file_path_finish = os.path.join(file_path1, file_name_finish)

# ส่งออก DataFrame เป็นไฟล์ Excel
df_left_join_4.to_excel(file_path_finish, index=False)